In [1]:
## BUSOBA 3332
## Project work 

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('data/hockey-data.csv')
df.head(n = 11)

,Year,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,...,SH.1,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%
0,2018-2019,1,Alex Ovechkin\ovechal01,33,WSH,LW,81,51,38,89,...,0,338,15.1,1695,20:55,41.0,223.0,1.0,4.0,20.0
1,2018-2019,2,Leon Draisaitl\draisle01,23,EDM,C,82,50,55,105,...,0,231,21.6,1852,22:35,26.0,57.0,677.0,663.0,50.5
2,2018-2019,3,John Tavares\tavarjo01,28,TOR,C,82,47,41,88,...,0,286,16.4,1565,19:05,53.0,59.0,833.0,694.0,54.6
3,2018-2019,4,Steven Stamkos\stamkst01,28,TBL,C,82,45,53,98,...,0,234,19.2,1501,18:18,40.0,72.0,506.0,447.0,53.1
4,2018-2019,5,Patrick Kane\kanepa01,30,CHI,RW,81,44,66,110,...,0,341,12.9,1822,22:29,18.0,21.0,1.0,4.0,20.0
5,2018-2019,6,Cam Atkinson\atkinca01,29,CBJ,RW,80,41,28,69,...,0,295,13.9,1550,19:23,25.0,27.0,24.0,52.0,31.6
6,2018-2019,7,Alex DeBrincat\debrial01,21,CHI,RW,82,41,35,76,...,0,220,18.6,1451,17:42,26.0,35.0,16.0,19.0,45.7
7,2018-2019,8,Nikita Kucherov\kucheni01,25,TBL,RW,82,41,87,128,...,0,246,16.7,1637,19:58,31.0,44.0,0.0,3.0,0.0
8,2018-2019,9,Nathan MacKinnon\mackina01,23,COL,C,82,41,58,99,...,0,365,11.2,1812,22:05,31.0,54.0,518.0,666.0,43.8
9,2018-2019,10,Connor McDavid\mcdavco01,22,EDM,C,78,41,75,116,...,1,240,17.1,1781,22:50,30.0,39.0,428.0,490.0,46.6
